In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

# Epipolar Geometry

In class we covered material for single and multi-camera systems. This notebook will help ground some of these geometric concepts.

## Camera Projection

First, we'll define some helper functions.

In [ ]:
def rot_x(t) :
    """Creates a 3D rotation matrix representing a rotation
    about the x-axis by t radians."""
    return np.array([
        [1,         0,          0],
        [0, np.cos(t), -np.sin(t)],
        [0, np.sin(t),  np.cos(t)]
    ])

def rot_y(t) :
    """Creates a 3D rotation matrix representing a rotation
    about the y-axis by t radians."""
    return np.array([
        [ np.cos(t), 0, np.sin(t)],
        [         0, 1,         0],
        [-np.sin(t), 0, np.cos(t)]
    ])

def rot_z(t) :
    """Creates a 3D rotation matrix representing a rotation
    about the z-axis by t radians."""
    return np.array([
        [np.cos(t), -np.sin(t), 0],
        [np.sin(t),  np.cos(t), 0],
        [        0,          0, 1]
    ])

def translation(x, y, z) :
    """Creates a 3D translation vector of shape 3x1."""
    return np.array([[x,y,z]]).T

def transformation(R, t) :
    """Creates a 3D homogeneous transformation matrix
    given a rotation matrix and a translation vector."""
    return np.concatenate([
        np.concatenate((R, t), axis=1),
        np.array([[0, 0, 0, 1]])
    ])

def intrinsic_matrix(fx, fy, ox, oy) :
    """Creates the 3x4 intrinsic matrix given camera parameters."""
    return np.array([
        [fx,  0, ox, 0],
        [ 0, fy, oy, 0],
        [ 0,  0,  1, 0],
    ])

Let's first define some points in 3D and visualize them.

In [ ]:
pts = np.array([
    [  26.121,  32.847, 1005.353, 1.],
    [ -44.027,  30.439,  973.450, 1.],
    [ -46.379, -13.096,  985.124, 1.],
    [  30.911,  34.686, 1030.866, 1.],
    [   4.990, -46.464, 1048.179, 1.],
    [  27.133, -28.476, 1014.759, 1.],
    [ -47.045, -45.281,  972.736, 1.],
    [  14.218, -26.843,  967.712, 1.],
    [  28.478,   2.358,  978.545, 1.],
    [  41.695,  36.650, 1009.917, 1.],
    [   7.024, -22.803,  964.057, 1.],
    [ -12.522,  45.107,  970.331, 1.],
    [  -7.957,  36.629,  974.575, 1.],
    [ -27.472, -30.949,  960.675, 1.],
    [  16.020, -17.619, 1047.176, 1.],
    [ -38.449, -48.483, 1045.377, 1.],
    [   8.596, -21.297,  982.135, 1.],
    [  44.768,   1.136, 1034.768, 1.],
    [ -13.129,   9.305, 1022.283, 1.],
    [   3.906,  32.476, 1027.867, 1.],
]).T
xs, ys, zs, _ = np.split(pts, 4, axis=0)

# Plot these points.
colors = plt.cm.rainbow(np.linspace(0, 1, pts.shape[1]))
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs=zs, s=20, c=colors)
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
plt.show()

(3 points) With the helper functions, do the following:
- Create an intrinsic matrix $M$ with parameters $f_x=f_y=1000$, $o_x=52.2$, $o_y=55.3$.
- Create an extrinsic matrix $H$ with no rotation and a translation $(x, y, z) = (42, 33, 1000)$.
- Using $M$ and $H$, create a projection matrix $P$. Print $P$.

In [ ]:
# TODO

np.set_printoptions(precision=5, suppress=True)
print(P)

(1 points) In the cell below, project the 3D points `pts` to 2D using this projection matrix and store the resulting pixel coordinates into `pxs`. Remember that the results are in homogeneous coordinates, so you will need to normalize each pixel such that $w=1$ for hoomogeneous coordinate $(u, v, w)$.

In [ ]:
pxs = 

# Plot these points
fig = plt.figure()
plt.scatter(pxs[0], pxs[1], s=10, c=colors)
plt.gca().invert_yaxis()
plt.xlabel('u'); plt.ylabel('v')
plt.axis('equal')
plt.show()

(1 point) Why did we invert the y axis here?

In [ ]:
# Type your answer below.
# 

Let's now compare the 2D projections with the 3D points viewd from the corresponding viewpoint.

In [ ]:
# Plot pts again but with a different viewpoint.
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs=zs, s=20, c=colors)
ax.view_init(elev=-90, azim=-90)
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
plt.show()

## Epipolar Geometry

Now that we're more confident with single camera geometry, let's try relate the geometries and the viewpoints of two cameras.

(1 point) For camera 1, create an intrinsic matrix with parameters $f_x=f_y=1000$, $o_x=35$, $o_y=55.3$, an extrinsic matrix with a y-axis rotation by -0.3 radians, and a translation vector of $(280, 33, 1000)$. Calculate the resulting projection matrix.

Project the same 3D points in the previous section into the view of this camera, and plot these pixel coordinates with the same colors by setting `c=colors`. Remember to invert the y-axis.

(1 point) Now do the same for 2 with intrinsic parameters $s_x=s_y=1000$, $o_x=35$, $o_y=55.3$, whereas the extrinsic matrix is defined by a y-axis rotation by 0.3 radians, and a translation vector of $[390, 33, 1000]$.

Now we have the pixel coordiantes in both images and their correspondence, we can solve for the fundamental matrix, which relates pixel coordinates from one image to another.

(2 points) Below, fill in the matrix $A$ following the slides.

(2 points) Solve for the fundamental matrix given that $Af=0$, and print out the fundamental matrix $F$. You should already know how to solve this after solving a similar problem in the notebook for module 6.

(1 point) Given the two camera calibration matrices and the fundamental matrix $F$, calculate the essential matrix $E$ and print the result.

Using the essential matrix $E$ we could recover $R$ and $t$ between the cameras using SVD. However, there are some nuances about the sign of results in this process and we could get 4 possible solutions for $R$. (For more details, check section 11.3 of the 2nd edition of [Richard Szeliski's book](https://szeliski.org/Book/).) For simplicity, in the cell below we will only show that one of the solutions matches the actual rotation we used to construct our camera projection matrices.

In [ ]:
# Print the actual rotation matrix we used.
R = rot_y(-0.6)
print(R)

In [ ]:
# Recover R from E.
u, s, vh = np.linalg.svd(E)

R1 =  u @ rot_z(np.pi/2) @ vh
print(np.allclose(R, R1))

R2 = -u @ rot_z(np.pi/2) @ vh
print(np.allclose(R, R2))

R3 =  u @ rot_z(-np.pi/2) @ vh
print(np.allclose(R, R3))

R4 = -u @ rot_z(-np.pi/2) @ vh
print(np.allclose(R, R4))